<a href="https://colab.research.google.com/github/pipecode-br/deep_learning_pytorch/blob/main/siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Etapa 1: Importar biblioteca

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
torch.__version__

In [7]:
!pip install roboflow
!pip install pylabel

     |████████████████████████████████| 145 kB 31.1 MB/s 
     |████████████████████████████████| 178 kB 57.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 138 kB 59.9 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=a8c935b51e3744cb9b71c1ebc4ff2fe6a452bf74a9f0b724edd1fbfdbb39d578
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=5fa4669a6d6355c5375ba6fd42c43c1464aaf9af801b38b90aaacfb19033558d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wge

In [ ]:
torch.manual_seed(123)

### Etapa 2: Construir modelo

In [ ]:
import torch
from torch import nn
from torchsummary import summary
import torch.nn.functional as F

class feature(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      # load resnet-18
      resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
      #primeiras camadas conv do resnet
      self.layer1 = nn.Sequential(
          resnet18.conv1,
          resnet18.bn1,
          resnet18.relu,
          resnet18.maxpool
        )
      self.layer2 = nn.Sequential(
          resnet18.conv1,
          resnet18.bn1,
          resnet18.relu,
          resnet18.maxpool
        )
      self.layer1.training = False
  def forward(self,X):
    X_ = self.layer1(X)
    return X_

class discriminator(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.flatten = nn.Flatten()
      size = 115200
      self.dense1 = nn.Linear(in_features=size,out_features=128)
      self.dense2 = nn.Linear(in_features=128,out_features=36)
      self.dense3 = nn.Linear(in_features=36,out_features=1)
      self.dropout = nn.Dropout(0.2)

  def forward(self,X1,X2):
    X1_ = self.flatten(X1)
    X2_ = self.flatten(X2)
    # X1 = X1.view(X1.size()[0], -1)
    # X2 = X2.view(X2.size()[0], -1)
    x = torch.cat((X1_,X2_), 1)
    x = self.dropout(F.leaky_relu(self.dense1(x),0.2))    
    x = self.dropout(F.leaky_relu(self.dense2(x),0.2))
    x = self.dropout(F.leaky_relu(self.dense3(x),0.2))
    return x

class siamese(nn.Module):
      def __init__(self) -> None:
          super().__init__()
          self.feature = feature()      
          self.discriminator = discriminator()

      def forward(self, X1, X2):          
          X1_ = self.feature(X1)
          X2_ = self.feature(X2)
          X = self.discriminator(X1_, X2_)
          return X

In [ ]:
from abc import ABC, abstractmethod

class DeepNeuralInterface(ABC,nn.Module):
    @abstractmethod
    def show(self):
      pass
    @abstractmethod
    def get_transforms(self):
      pass
      
class TrainSiameseInterface(ABC):
    @abstractmethod
    def train(self,net:nn.Module,X)->None:
      pass

    @abstractmethod
    def train_epoch(self,net:nn.Module,X)->None:
      pass

class teste(TrainSiameseInterface):
  def __init__(self,device) -> None:
      super().__init__()
      self.device = device

  def train(self,net,train_loader,epochs=15)->None:
    optimizer = optim.Adam(net.parameters(),lr=0.002)
    criterion = nn.BCEWithLogitsLoss()
    net.to(device)
    for epoch in range(epochs):
      D_running_loss = 0
      G_running_loss = 0

      for i,(imagens_reais,_) in enumerate(train_loader):

        batch_size= imagens_reais.size(0)
                   
        imagens_reais = imagens_reais * 2-1       
        imagens_reais = imagens_reais.to(self.device)

        #treinamento com reais
        outputs_reais = net.forward(imagens_reais,imagens_reais)
        labels_reais = (torch.ones(batch_size)*0.9).to(device)
        D_loss_reais = criterion(outputs_reais.view(*labels_reais.shape),labels_reais)      

        #treinamento com falso  
        imagens_ruido = torch.flip(imagens_reais, [0,1])
        outputs_falsos = net.forward(imagens_reais,imagens_ruido)
        labels_falsos= torch.zeros(batch_size).to(device)
        D_loss_falsos = criterion(outputs_reais.view(*labels_falsos.shape),labels_falsos)

        D_loss = D_loss_reais+ D_loss_falsos
        D_loss.backward()
        optimizer.step()

        D_running_loss += D_loss.item()
        # print('train ',i,imagens_reais.shape,D_loss_reais,D_loss_falsos)

      D_running_loss /=len(train_loader)
      print('epoch -> {} loss -> {}'.format(epoch,D_running_loss)) 


  def train_epoch(self,net,X)->None:
    pass




In [ ]:
#criar transformador
#transform = transforms.ToTensor()
preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(120),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
import torch
from torchvision import transforms
import numpy as np
import pandas as pd
from skimage import io
from skimage.transform import resize
from skimage.draw import rectangle
from roboflow import Roboflow
from pylabel import importer
from PIL import Image
class FibersDataset(torch.utils.data.Dataset):
    """Fibers dataset."""

    def __init__(self,transform=None):
      rf = Roboflow(api_key="JhqX7HlUL57cmzbBqIav")
      project = rf.workspace().project("fibberpaper")
      dataset = project.version(1).download("yolov5")
      dataset = importer.ImportYoloV5(path='/content/FibberPaper-1/test/labels', path_to_images='/content/FibberPaper-1/test/images')
      self.mapa,self.data = self.extractFibersOfImagens(dataset.df)
      self.sourceTransform= transform
      self.rootDir = '.'


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        image = Image.fromarray(self.data[idx])
        if self.sourceTransform:
            image = self.sourceTransform(image)

        class_id = torch.tensor([0])
        return image,class_id


    #extrar o conjunto de pixel das fibras contidas no boundbox descrito no dataframe
    def extractFibersOfImagens(self,df):
      mapa = {}
      fibers = []
      #para cada linha 
      for index, row in df.iterrows():
        if "idoc" in row.img_filename: 
          continue
        #ler a imagem e salva no map
        path_full='{}/{}'.format(row.img_folder,row.img_filename)
        if mapa.get(row.img_filename) is None:
          img = io.imread(path_full)
          mapa[row.img_filename]=img
        #recorta o box da fibra
        x1,y1,x2,y2 = int(row.ann_bbox_xmin),int(row.ann_bbox_ymin),int(row.ann_bbox_xmax),int(row.ann_bbox_ymax)
        fibers.append(mapa[row.img_filename][y1:y2,x1:x2])
        #normaliza a imagem com o transforms
        #tenta remover o fundo
      return mapa,np.array(fibers)


train = FibersDataset(transform=preprocess)
# criar loader
train_loader = torch.utils.data.DataLoader(train,batch_size=16)

## Criação dos objetos

In [ ]:
net = siamese()
param = net.parameters()
optimizer = optim.Adam(param,lr=0.002)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

In [ ]:
teste(device=device).train(net=net,train_loader=train_loader,epochs=500)

In [ ]:
img1 =(3 , 120, 120)
summary(net,[img1,img1])

### Separa primeiras camadas

### visualiza resultado

In [ ]:
import matplotlib.pyplot as plt

def plot_result(input_image,conv_unif):
  plt.style.use('classic')
  plt.imshow(input_image.transpose(2, 0))
  plt.show()
  plt.imshow(conv_unif)
  plt.show()


## Testa sub modelo em um exemplo real

In [ ]:
for input_image,_ in train:
  # input_image,_ = next(iter(train))
  input_batch = input_image.unsqueeze(0) 
  print(input_batch.shape)

  model1 = feature()
  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
      input_batch = input_batch.to('cuda')
      model1.to('cuda')

  with torch.no_grad():
      output = model1(input_batch)
      out1=output.cpu().numpy()
  conv_unif = np.sum(out1[0],axis=0)/out1[0].shape[0]
  print(conv_unif.shape)
  plot_result(input_image,conv_unif)

# testar rede siamesa


### concatena features da camada de saida

In [ ]:
train_batch = next(iter(train_loader))
for img,_ in train_loader:
  print(t)

In [ ]:

for input_image,_ in train:
  # input_image,_ = next(iter(train))
  input_batch = input_image.unsqueeze(0) 
  print(input_batch.shape)
  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
      input_batch = input_batch.to('cuda')
      net = net.to('cuda')
  with torch.no_grad():
      output = net(input_batch,input_batch)
      out1=output.cpu().numpy()
      print(out1)